In [1]:
import sys
import numpy as np
import mxnet as mx
import tensorflow as tf
from datascience import *
import math as math

In [46]:
batch_size = 100
num_it = 10000
num_words = 4976
num_genre = 2

In [ ]:
lyrics = Table().read_table('lyrics3.csv')
# altered = lyrics.select(np.arange(2, lyrics.num_columns))
# altered.to_csv('genreAndProp.csv')
table = Table().read_table('vectorLyrics.csv')
# propsOnly = lyrics.select(np.arange(3, lyrics.num_columns))
# genresOnly = Table().with_columns('Country', lyrics.apply(lambda x: ((int) (x=='Country')) , 'Genre'), 'Hip-hop', lyrics.apply(lambda x: ((int) (x=='Hip-hop')) , 'Genre'))
# propsOnly.to_csv('lyricsProps.csv')
# genresOnly.to_csv('lyricsGenres.csv')

Our input vectors are of length 4816, because we have 4816 words.
Our output vectors are of length 2, and are a softmax. 1 in the first position means country, 1 in the second position means hip hop.

In [4]:
# filename_queue = tf.train.string_input_producer(["lyricsProps.csv", "lyricsGenres.csv"], num_epochs = 6)

# reader = tf.TextLineReader(skip_header_lines=1)
# key, value = reader.read(filename_queue)

# # Default values, in case of empty columns. Also specifies the type of the
# # decoded result. 
# record_defaults = [[0] for _ in range(4819)]
# reading = tf.decode_csv(value, record_defaults=record_defaults)
# features = tf.stack(reading[0:len(reading)-2])
# genres = tf.stack(reading[len(reading)-2:])

In [5]:
# def create_file_reader_ops(filename_queue):
#     reader = tf.TextLineReader(skip_header_lines=1)
#     _, csv_row = reader.read(filename_queue)
#     record_defaults = [[0] for _ in range(4819)]
#     reading = tf.decode_csv(csv_row, record_defaults=record_defaults)
#     features = tf.stack(reading[0:len(reading)-2])
#     genres = tf.stack(reading[len(reading)-2:])
#     return features, genres

In [6]:
# directory = "vectorLyrics.csv"
# filename_queue = tf.train.string_input_producer(
#     tf.train.match_filenames_once(directory),
#     shuffle=True)

# # Each file will have a header, we skip it and give defaults and type information
# # for each column below.
# line_reader = tf.TextLineReader(skip_header_lines=1)

# _, csv_row = line_reader.read(filename_queue)

# # Type information and column names based on the decoded CSV.
# record_defaults = [[0.0] for _ in range(4817)]+[[0], [0]]
# readings = tf.decode_csv(csv_row, record_defaults=record_defaults)
# # Turn the features back into a tensor.
# features = tf.stack(readings[:4817])
# genre = readings[4817:]

# with tf.Session() as sess:
#     tf.global_variables_initializer().run()

#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     # We do 10 iterations (steps) where we grab an example from the CSV file. 
#     for iteration in range(1, 11):
#         # Our graph isn't evaluated until we use run unless we're in an interactive session.
#         example, label = sess.run([features, genre])

#         print(example, label)
#     coord.request_stop()
#     coord.join(threads)

In [30]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [43]:
# x = tf.placeholder(tf.float32, [None, 4976])
# W1 = tf.Variable(tf.zeros([4976, 1000]))
# b1 = tf.Variable(tf.zeros([1000]))
# y1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# W2 = tf.Variable(tf.zeros([1000, 10]))
# b2 = tf.Variable(tf.zeros([10]))
# y2 = tf.sigmoid(tf.matmul(y1, W2) + b2)

# W3 = tf.Variable(tf.zeros([10, 2]))
# b3 = tf.Variable(tf.zeros([2]))
# y3 = tf.nn.softmax(tf.matmul(y2, W3) + b3)


x = tf.placeholder(tf.float32, [None, num_words])
# W = tf.Variable(tf.zeros([4976, 2]))
W = tf.Variable(tf.truncated_normal([num_words, num_genre],
                                    stddev=1./math.sqrt(num_words)))
b = tf.Variable(tf.zeros([num_genre]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
# y = tf.transpose(y)

In [44]:
y_ = tf.placeholder(tf.float32, [None, num_genre])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [7]:
arr = np.array(table.row(0))
trainX = np.array(np.array([arr[:num_words]]))
trainY = np.array(np.array([arr[num_words:]]))
trainY1 = np.array(np.array([arr[num_words:num_words+1]]))
for i in range(1, table.num_rows):
    arr = np.array(table.row(i))
    trainX = np.vstack((trainX, np.array([arr[:num_words]])))
    trainY = np.vstack((trainY, np.array([arr[num_words:]])))

In [45]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
count = 0
while count < num_it:
    try:
        sample = table.sample(batch_size)
        arr = np.array(sample.row(0))
        features = np.array(np.array([arr[:num_words]]))
        genre = np.array(np.array([arr[num_words:]]))
        for i in range(1, back_size):
            arr = np.array(table.row(i))
            features = np.vstack((features, np.array([arr[:num_words]])))
            genre = np.vstack((genre, np.array([arr[num_words:]])))
#         features = np.array([arr[:4976]])
#         genre = np.array([arr[4976:]])
        feed_dict={x: features, y_: genre}
        example_data = sess.run(train_step, feed_dict)
#         print(genre)
#         print(example_data)
    except tf.errors.OutOfRangeError:
        break
    print('training session' + str(count))
    count+=1

training session0
training session1
training session2
training session3
training session4
training session5
training session6
training session7
training session8
training session9
training session10
training session11
training session12
training session13
training session14
training session15
training session16
training session17
training session18
training session19
training session20
training session21
training session22
training session23
training session24
training session25
training session26
training session27
training session28
training session29
training session30
training session31
training session32
training session33
training session34
training session35
training session36
training session37
training session38
training session39
training session40
training session41
training session42
training session43
training session44
training session45
training session46
training session47
training session48
training session49
training session50
training session51
training session52
tra

In [94]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: trainX , y_: trainY }))

RuntimeError: Attempted to use a closed Session.

In [75]:
holdout = Table.read_table('holdout.csv')
arr = np.array(table.row(0))
testX = np.array(np.array([arr[2:]]))
for i in range(1, holdout.num_rows):
    arr = np.array(holdout.row(i))
    testX = np.vstack((testX, np.array([arr[2:]])))

In [50]:
hCoun = np.array([])
hHip = np.array([])
prediction=tf.argmax(y,1)
pred_arr = np.array(prediction.eval(feed_dict={x: trainX}, session=sess))
print ("predictions", pred_arr)

predictions [1 1 1 ..., 1 0 0]


In [49]:
probabilities=y
print ("probabilities", probabilities.eval(feed_dict={x: testX}, session=sess))

probabilities [[ 0.56033701  0.43966299]
 [ 0.60368323  0.39631686]
 [ 0.16368942  0.83631057]
 [ 0.60886723  0.39113277]
 [ 0.51326084  0.48673916]
 [ 0.58933133  0.4106687 ]
 [ 0.6750949   0.3249051 ]
 [ 0.70720845  0.29279155]
 [ 0.52718127  0.47281873]
 [ 0.8633219   0.13667807]
 [ 0.49579677  0.5042032 ]
 [ 0.76771539  0.23228459]
 [ 0.82075053  0.17924948]
 [ 0.713377    0.286623  ]
 [ 0.39995271  0.60004729]
 [ 0.53810602  0.46189401]
 [ 0.78809071  0.21190926]
 [ 0.83616757  0.16383244]
 [ 0.38974646  0.61025357]
 [ 0.51183659  0.48816341]
 [ 0.43994239  0.56005764]
 [ 0.53089911  0.46910092]
 [ 0.29306546  0.70693451]
 [ 0.66457224  0.33542782]
 [ 0.80816084  0.19183908]
 [ 0.48369509  0.51630491]
 [ 0.14931618  0.85068387]
 [ 0.68490708  0.31509283]
 [ 0.72373569  0.27626437]
 [ 0.60584861  0.39415136]
 [ 0.5258792   0.47412086]
 [ 0.64171535  0.35828465]
 [ 0.53772986  0.46227014]
 [ 0.42901531  0.57098472]
 [ 0.51684844  0.48315164]
 [ 0.57439846  0.42560151]
 [ 0.51904148 

In [77]:
holdout = Table().read_table('holdout_attributes.csv')
holdout_attr = holdout.drop('Id')
arr = np.array(holdout_attr.row(0))
holdX = np.array(np.array([arr]))
for i in range(1, holdout_attr.num_rows):
    arr = np.array(holdout_attr.row(i))
    holdX = np.vstack((holdX, np.array([arr])))

In [63]:
def create_competition_submission(predictions, filename='my_submission.csv'):
    """
    Create a submission CSV for the Kaggle competition.
    
    Inputs:
      predictions - list or array of your predictions (Generated as in Question 3.3.1.)
    """
    Table().with_columns('Id', np.arange(len(predictions)), 'Predictions', predictions).to_csv(filename)
    print('Created', filename)

In [83]:
holdout_pred = np.array(prediction.eval(feed_dict={x: holdX}, session=sess))
print(holdout_pred)
create_competition_submission(holdout_pred, 'submission1.csv')

ValueError: Cannot feed value of shape (100, 4817) for Tensor 'Placeholder_9:0', which has shape '(?, 4976)'

In [82]:
submission = Table().read_table('submission1.csv')
submission

Id,Predictions
0,1
1,1
2,1
3,0
4,0
5,0
6,0
7,1
8,0
9,1


In [89]:
holdout.num_columns
orgHold = Table().read_table('holdout.csv')
holdout.num_columns

4818

In [87]:
lyrics2 = Table().read_table('lyrics2.csv')
lyrics2.num_columns

4979

i         | the       | you       | to         | and        | a          | me        | it         | not       | in         | my         | is         | of         | your       | that       | do         | on         | are        | we         | am         | will       | all        | for        | no         | be         | have       | love       | so         | know       | this       | but        | with       | what       | just       | when      | like       | now        | que       | time       | can        | come       | de        | there      | go         | up         | oh         | la       | one        | they       | out        | down       | get        | she        | was        | see        | if         | got        | never      | from       | he         | feel       | want       | let        | make       | way        | say        | take       | would      | as         | ca         | day        | at         | babi       | away       | life       | yeah       | y        | back       | by         | her        | heart      | here       | how        | could      | night      | need       | our        | look       | where      | en         | eye  | thing      | world      | more       | caus       | gonna      | die        | right      | been       | tell       | think     | un   | who        | el        | through    | man        | live       | again      | give       | too        | onli       | te       | tri        | tu   | or         | whi        | se   | keep       | dream      | well       | mind       | an         | wo         | still      | us         | his        | long       | mi   | girl       | wanna      | find       | fall      | around     | good       | about      | some       | over       | littl      | turn       | hand       | then       | noth       | light      | said       | call       | everi      | alway      | were       | did        | into       | e    | wait | leav       | home | had        | hold | run        | es         | walk       | ever       | them       | lie       | face     | end        | hear       | gone       | head       | believ     | everyth | ya         | has  | cri       | alon       | yo         | hey        | their      | le   | someth     | si        | insid | du   | befor      | o    | word       | stay       | place      | much       | chang      | ich  | god        | stand      | these      | better     | last       | friend | soul | anoth      | je   | left       | new        | him        | than       | stop       | tonight | burn | sun        | off        | por      | lo         | play       | thought    | sing | show       | lost | peopl      | made       | break      | und  | start      | should     | realli     | chorus | hard | sky  | old        | da   | even | boy  | pleas      | song       | care       | without | hope | watch      | pain       | amor     | move       | wrong      | put        | free       | gotta      | name | et   | same       | blue | con      | danc | talk       | seem | own  | mine       | dark      | na   | far  | tear | true | fire       | use        | dead | bring      | other | sin      | someon | myself | forev | close     | sleep | rain       | those      | must | una  | les  | sweet      | mean       | fight | rememb     | star       | while      | kiss | kill       | high | breath | doe        | yes  | al   | wish       | che  | real       | smile      | two        | cold       | lord | year       | open       | il   | fear       | di   | done       | side       | fli  | black      | line       | rock       | blood      | lose | found     | onc        | non  | door | bad  | ooh  | wonder | ti   | como       | enough     | para       | sound | togeth | mayb       | los        | help       | shine      | mani       | heaven     | becaus     | work       | touch | behind     | yourself | today      | big  | might | death | hate       | miss | ride       | street     | came       | ask        | hide       | everybodi | save       | till | may  | 

In [93]:
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

Model saved in file: /tmp/model.ckpt


4820